# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving informaton out of natural language (could be seqeuences text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

> 📖 **Resource:** See all course materials, resources and extra-curriculum for this notebook on GitHub: https://github.com/mrdbourke/tensorflow-deep-learning/

Video N° 225: Preparing a notebook for our first NLP with TensorFlow project

## Check for GPU

In [1]:
import numpy as np

from MachineLearningUtils.system_setup.gpu import activate_gpu
activate_gpu()

2024-04-26 12:02:59.653781: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 12:02:59.653819: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 12:02:59.683422: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-26 12:02:59.736336: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


1 Physical GPUs, 1 Logical GPU


2024-04-26 12:03:02.477465: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-26 12:03:02.759279: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-26 12:03:02.759584: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
!nvidia-smi -L

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
GPU 0: NVIDIA GeForce RTX 3050 (UUID: GPU-ee6cd8ff-34bf-dc58-2edb-4f5120aaec55)


## Get helper functions

In [3]:
from MachineLearningUtils.data_acquisition.data_downloader import *
from MachineLearningUtils.data_visualization.augmentation_effects import *
from MachineLearningUtils.data_visualization.evaluation_metrics import *
from MachineLearningUtils.data_visualization.image_visualization import *
from MachineLearningUtils.data_visualization.model_learning_curves import *
from MachineLearningUtils.data_visualization.prediction_visualization import *
from MachineLearningUtils.training_utilities.model_callbacks import *
from MachineLearningUtils.training_utilities.transfer_learning import *

## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as diaster or not diaster).

See the original source here: https://www.kaggle.com/c/nlp-getting-started

In [4]:
url = "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"
download_data(url=url, file_path="nlp_getting_started.zip", extract=True)

The file nlp_getting_started.zip already exists.
Extracting nlp_getting_started.zip as ZIP...
nlp_getting_started.zip has been extracted to current directory.


Video N°226: Becoming one with the data and visualising a text dataset

## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would be to use Python: https://realpython.com/read-write-files-python/

But I prefer to get visual straight away.

So another way to do this is to use pandas...

In [5]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of each class?
train_df_shuffled.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df) - 5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target= row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 1 (real disaster)
Text:
@SatanaOfHell ever seen by far. A dreamy look came over his face and his ears flattened as he started to walk slowly toward her.* You'd-

---

Target: 1 (real disaster)
Text:
Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/VBsj8tniv1

---

Target: 0 (not real disaster)
Text:
I'm not a Drake fan but I enjoy seeing him body-bagging people. Great marketing lol.

---

Target: 1 (real disaster)
Text:
Suicide bomber kills 15 in Saudi security site mosque - Reuters via World - Google News - Wall ... http://t.co/nF4IculOje

---

Target: 1 (real disaster)
Text:
My brother-n-law riooooos got the call to head up north and fight the wild fires. Dudes a beast atÛ_ https://t.co/463P0yS0Eb

---


Video N°227: Splitting data into training and validation sets

### Split data into training and validation sets

We want to be able to see how our model is performing on unseen data whilst it trains.

And because the testing dataset doesn't have labels, we'll have to create a validation dataset to evaluate on (the model won't see the validation dataset during training so we can use its samples and labels to evaluate our model's performance).

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation split
                                                                            random_state=42)

In [13]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

Video N°228: Converting text data to numbers using tokenisation and embeddings (overview)
Video N°229: Setting up a TensorFlow TextVectorization layer to convert text to numbers

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenziation - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

In [15]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=10000, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be?
                                    pad_to_max_tokens=True)

In [17]:
train_sentences[0].split(), len(train_sentences[0].split())

(['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet'], 7)

In [18]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [19]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

Video N°230: Mapping the TextVectorization layer to text data and turning it into numbers

In [20]:
# Fit the text vectorizer instance to the training data using the adapt() method
text_vectorizer.adapt(train_sentences)

In [21]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
@PrincessDuck last week wanted the 6th sense to get blown up so far so good. James could win but he's a huge target and will be gone soon.        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[9574,  139,  561,  974,    2, 4171, 1177,    5,   52,  468,   27,
          28,  685,   28,  136]])>

In [23]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Most common words in vocab: {top_5_words}")
print(f"Least common words in vocab: {bottom_5_words}")

Most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
Least common words in vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [24]:
# how long is our vocab?
len(words_in_vocab)

10000

Video N°231: Creating an Embedding layer to turn tokenised text into embedding vectors

## Creating an Embedding using an Embedding Layer

To make our embedding, we're going to use TensorFlow's embedding layer:
[https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding)
The parameters we care most about for our embedding layer:
- `input_dim` = the size of our vocabulary
- `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
- `input_length` = length of the sequences being passed to the embedding layer

In [25]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set the input shape
                             output_dim=128, # set the size of the embedding vector
                             embeddings_initializer="uniform", # default, initialize embedding vectors randomly
                             input_length=max_length) # how long is each input
embedding

In [26]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f'Original text:\n {random_sentence}')
print(f'\n\nEmbedded version:')

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Runaway Minion Causes Traffic Collapse in Dublin http://t.co/u2Kwof3wtj


Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00550646, -0.04917727,  0.01549926, ..., -0.01024961,
          0.02381234,  0.03026136],
        [-0.03436272, -0.0145077 ,  0.0317042 , ...,  0.00794063,
         -0.02100012,  0.02503968],
        [ 0.01950933,  0.03169134, -0.02611487, ...,  0.01726953,
         -0.02312267,  0.02902061],
        ...,
        [-0.02583178, -0.03553148, -0.03554224, ..., -0.04668079,
         -0.03468776,  0.00903301],
        [-0.02583178, -0.03553148, -0.03554224, ..., -0.04668079,
         -0.03468776,  0.00903301],
        [-0.02583178, -0.03553148, -0.03554224, ..., -0.04668079,
         -0.03468776,  0.00903301]]], dtype=float32)>

In [27]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.00550646, -0.04917727,  0.01549926,  0.03699687,  0.00409739,
        -0.02820637, -0.04731511, -0.0111442 ,  0.03491685,  0.01096799,
        -0.00482909, -0.01479162, -0.04238294,  0.04745207, -0.01825277,
         0.03378871,  0.0096984 ,  0.01250324, -0.04644053,  0.02253062,
         0.03956306, -0.00630814,  0.04621444,  0.03919097,  0.01205512,
        -0.03027948,  0.03953877, -0.02110714, -0.02828534, -0.00983946,
        -0.03750581, -0.01811235,  0.00902592,  0.00389408,  0.04717649,
         0.03801909, -0.0338936 ,  0.04097966, -0.04807104, -0.01801399,
         0.04736641,  0.0145645 , -0.01578894, -0.02512347, -0.00857364,
        -0.04228088,  0.01462798, -0.01756846,  0.03941492, -0.02230245,
         0.01679957, -0.03221705,  0.02691228,  0.01181713, -0.03911083,
         0.04477019,  0.01402367,  0.04069215, -0.03707717, -0.04772345,
        -0.03592135,  0.02227445,  0.01458988, -0.01986423,  0.02568338,
  

Video N°232: Discussing the various modelling experiments we're going to run

## Modelling a text dataset (setting up our modelling experiments)

Now we've a got way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline) - got this from here: [https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Word Embedding (feature extractor) (using transfer learning for NLP)
* Model 7: Same as model 6 but using 10% of data

How are we going to approach all of these?
Use the standard steps in modelling with tensorflow:
* Create a model
* Build a model
* Fit a model
* Evaluate our model

Video N°233: Model 0:Building a basline model to try and improve upon

### Model 0: Getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

> 🔑 **Note:** It's common practice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if you can improve upon them.

In [28]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf - https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
    ("clf", MultinomialNB()) # model the text converted to numbers
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

CPU times: user 270 ms, sys: 7.54 ms, total: 277 ms
Wall time: 280 ms


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [30]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [31]:
train_labels

array([0, 0, 1, ..., 1, 1, 0])

Video N°234: Creating a function to track and evaluate our model's results

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time, however, this will be cumbersome and could easily be fixed with a function

Let's create one to compare our model's predictions with the truth labels using the following metrics:
 
* Accuracy
* Precision
* Recall
* F1-score

For a deep overview of many different evaluation methods, see the Sklearn documentation: [https://scikit-learn.org/stable/modules/model_evaluation.html](https://scikit-learn.org/stable/modules/model_evaluation.html)

In [32]:
# Get baseline results with personal function
baseline_results = evaluate_classification_metrics(y_true=val_labels, y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

Video N°235: Model1: Building, fitting and evaluating our first deep model on text data

### Model 1: A simple dense model (feed-forward neural network)

In [33]:
# Create tensorboard callback (need to create a new one for each model)
# import personal 'create_tensorboard_callback' function

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [34]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [35]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers 
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation function
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [36]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [37]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
history_1 = model_1.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240426-120306
Epoch 1/5


2024-04-26 12:03:07.814667: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f9ef88abc30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-26 12:03:07.814703: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050, Compute Capability 8.6
2024-04-26 12:03:07.862393: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-26 12:03:07.945534: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1714125788.052850   13826 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


215/215 [==============================] - 8s 31ms/step - loss: 0.6118 - accuracy: 0.6935 - val_loss: 0.5371 - val_accuracy: 0.7546
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.4416 - accuracy: 0.8168 - val_loss: 0.4702 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3473 - accuracy: 0.8619 - val_loss: 0.4580 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2843 - accuracy: 0.8895 - val_loss: 0.4610 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2374 - accuracy: 0.9111 - val_loss: 0.4812 - val_accuracy: 0.7874


In [38]:
# Evaluate model 1
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4812 - accuracy: 0.7874


[0.48116424679756165, 0.787401556968689]

In [39]:
# Look at the first 10 predictions
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.3762395 ],
       [0.7522857 ],
       [0.9972192 ],
       [0.10433523],
       [0.11103633],
       [0.93116134],
       [0.90806293],
       [0.9954508 ],
       [0.95062214],
       [0.23266819]], dtype=float32)

In [40]:
# Convert model prediction
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [41]:
# Caluclate our model_1 results
model_1_results = evaluate_classification_metrics(y_true=val_labels,
                                                  y_pred=model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7927656871284042,
 'recall': 0.7874015748031497,
 'f1': 0.7842639469124086}

In [42]:
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

Nous n'avons donc pas de meilleurs résultats avec model_1 comparé à model_0

Video N°236: Visualising our model's learned word embeddings with TensorFlow's projector tool

### Visualizing learned embeddings

To further understand word embeddings, let's visualize them, to do so, we'll get the weights matrix (embedding matrix) from our embedding layer and visualize it using the Embedding project tool, see the TensorFlow guide for more: https://www.tensorflow.org/tutorials/text/word_embeddings

In [43]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [44]:
len(words_in_vocab)

10000

In [45]:
max_vocab_length

10000

In [46]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [47]:
# Get the weight matrix of embedding layer
# (these are the numerical representations of each token in our training data, which have been learned for 5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape) # same size as vocab size and embedding_dim (output_dim of our embedding layer)

(10000, 128)


Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TensorFlow has a handy tool called projector: [https://projector.tensorflow.org/](https://projector.tensorflow.org/)

And TensorFlow also has an incredible guide on word embeddings themselves: [https://www.tensorflow.org/text/guide/word_embeddings](https://www.tensorflow.org/text/guide/word_embeddings)

In [48]:
# Create embedding files (we got this from TensorFlow's word embedding documentation)
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

If you are running this tutorial in Colaboratory, you can use the following snippet to download these files to your local machine (or use the file browser, View -> Table of contents -> File browser).
```python
# Download files from Colab to upload projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass
```

Our visual word embeddings might not look like much to us, but they help our model understand the relationships between words.

For more on a popular type of word embedding and more visual explanations check out the illustrated word2vec: [https://jalammar.github.io/illustrated-word2vec/](https://jalammar.github.io/illustrated-word2vec/)

Video N°237: High-level overview of Recurrent Networks (RNNs) + where to learn more

## Recurrent Neural Networks (RNN's)

RNN's are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

**Resources:** If you want an overview of the internals of a recurrent neural network, see the following:
- MIT's sequence modelling lecture [https://www.youtube.com/watch?v=qjrad0V0uJE](https://www.youtube.com/watch?v=qjrad0V0uJE)
- Chris Olah's intro to LSTMs: [https://colah.github.io/posts/2015-08-Understanding-LSTMs/](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- Andrej Karpathy's the unreasonable effectiveness of recurrent neural networks: [https://karpathy.github.io/2015/05/21/rnn-effectiveness/](https://karpathy.github.io/2015/05/21/rnn-effectiveness/)

Video N°238: Model 2: Building, fitting and evaluating our first TensorFlow RNN model (LSTM)

### Model 2:LSTM

LSTM = long short term memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this:
```
Input (text) -> Tokenize -> Embedding -> Layers (RNN/dense) -> Output (label probability)
```

In [49]:
# Create an LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
print(f"input: A 3D tensor with shape [batch, timesteps, feature]")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")
#x = layers.LSTM(units=64, return_sequences=True)(x) # when you're stacking RNN cells together, you need to return_sequences=True
#print(x.shape)

x = layers.LSTM(units=64)(x)
print(f"x: A 2D tensor with shape [batch, features]")
print(f"After LSTM cell: {x.shape}")
#x = layers.Dense(units=64, activation="relu")(x)
#print(f"After Dense layer: {x.shape}")


outputs = layers.Dense(1, activation="sigmoid")(x)#binary data -> sigmoid
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

input: A 3D tensor with shape [batch, timesteps, feature]
After embedding: (None, 15, 128)
x: A 2D tensor with shape [batch, features]
After LSTM cell: (None, 64)


**Resource:** [https://ml-cheatsheet.readthedocs.io/en/latest/activation_functions.html#tanh](https://ml-cheatsheet.readthedocs.io/en/latest/activation_functions.html#tanh)

In [50]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [51]:
# Compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [52]:
# Fit the model
history_2 = model_2.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="Model_2_LSTM")])

Saving TensorBoard log files to: model_logs/Model_2_LSTM/20240426-120322
Epoch 1/5
215/215 [==============================] - 10s 34ms/step - loss: 0.2266 - accuracy: 0.9172 - val_loss: 0.6108 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1563 - accuracy: 0.9415 - val_loss: 0.5978 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1271 - accuracy: 0.9526 - val_loss: 0.7083 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1043 - accuracy: 0.9597 - val_loss: 0.8021 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0841 - accuracy: 0.9654 - val_loss: 0.9909 - val_accuracy: 0.7625


In [53]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[8.1236027e-03],
       [8.0996096e-01],
       [9.9973887e-01],
       [9.1187716e-02],
       [6.0168048e-04],
       [9.9959511e-01],
       [4.8256549e-01],
       [9.9987841e-01],
       [9.9974924e-01],
       [4.5240116e-01]], dtype=float32)

In [54]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 0.], dtype=float32)>

In [55]:
# Calculate model 2 results
model_2_results = evaluate_classification_metrics(y_true=val_labels,
                                                  y_pred=model_2_preds)
model_2_results

{'accuracy': 76.24671916010499,
 'precision': 0.7657677169674478,
 'recall': 0.7624671916010499,
 'f1': 0.7593667569927582}

In [56]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [57]:
np.array(list(model_2_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

Nous n'avons donc pas de meilleurs résultats avec model_2 comparé à model_0

Video N°239: Model 3: Building, fitting and evaluating our first TensorFlow RNN model(LSTM)

### Model 3: GRU

Another popular and effective RNN component is the GRU or 'gated recurrent unit'.

The GRU cell has similar features to an LSTM cell but has less parameters.


In [58]:
# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype=tf.string)
print(f"input: A 3D tensor with shape [batch, timesteps, feature]")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")

x = layers.GRU(units=64)(x)
print(f"After GRU 64 layer: {x.shape}")

#x = layers.GRU(units=64, return_sequences=True)(x) # if you want to stack recurrent layers on top of each other, you need return_sequences=True
#print(f"After GRU 64 layer: {x.shape}")

#x = layers.LSTM(units=64, return_sequences=True)(x)
#print(f"Ater LSTM 64 layer: {x.shape}")

# x = layers.GRU(units=64)(x)
# print(f"After GRU 64 layer: {x.shape}")

#x = layers.Dense(units=64, activation="relu")(x)
#print(f"After Dense 64 layer: {x.shape}")

outputs = layers.Dense(1, activation="sigmoid")(x)
print(f"After Dense 1 layer: {x.shape}")
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

input: A 3D tensor with shape [batch, timesteps, feature]
After embedding: (None, 15, 128)
After GRU 64 layer: (None, 64)
After Dense 1 layer: (None, 64)


In [59]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1317313 (5.03 MB)
Trainable params: 1317313 (5.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________

In [60]:
# Compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [61]:
# Fit the model
model_3_history = model_3.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20240426-120340
Epoch 1/5
215/215 [==============================] - 9s 35ms/step - loss: 0.1647 - accuracy: 0.9324 - val_loss: 0.6594 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0888 - accuracy: 0.9682 - val_loss: 0.7708 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0718 - accuracy: 0.9733 - val_loss: 1.0759 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0619 - accuracy: 0.9750 - val_loss: 1.1255 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0558 - accuracy: 0.9746 - val_loss: 1.1308 - val_accuracy: 0.7730


In [62]:
# Make some predictions with our GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 1s 2ms/step


array([[3.9600655e-03],
       [7.2909802e-01],
       [9.9990726e-01],
       [7.9756491e-02],
       [1.5480012e-04],
       [9.9962437e-01],
       [8.9192325e-01],
       [9.9995649e-01],
       [9.9989808e-01],
       [7.9792655e-01]], dtype=float32)

In [63]:
# Convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [64]:
# Calculate model 3 results
model_3_results = evaluate_classification_metrics(y_true=val_labels,
                                                  y_pred=model_3_preds)
model_3_results

{'accuracy': 77.29658792650919,
 'precision': 0.7742755789705,
 'recall': 0.7729658792650919,
 'f1': 0.7710949612836401}

In [65]:
np.array(list(model_3_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

Nous n'avons donc pas de meilleurs résultats avec model_3 comparé à model_0

Video N°240: Model 4: Building, fitting and evaluating a bidirectional RNN model

### Model 4: Bidirectional RNN

Normal RNN's go from left to right (just like you'd read an English sentence) however, a bidirectional RNN goes from right to left as well as left to right.

In [66]:
# Building a bidirectional RNN in TensorFlow
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layer=layers.LSTM(units=64, return_sequences=True))(x)
x = layers.Bidirectional(layer=layers.LSTM(units=64))(x)
print(f"64 neurons give 128 parameters because they are bidirectional: {x.shape}")
outputs = layers.Dense(units=1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

64 neurons give 128 parameters because they are bidirectional: (None, 128)


In [67]:
# Get a summary
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1378945 (5.26 MB)
Trainable par

In [68]:
# Compile model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [69]:
# Fit the model
model_4_history = model_4.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20240426-120358
Epoch 1/5
215/215 [==============================] - 10s 35ms/step - loss: 0.1121 - accuracy: 0.9661 - val_loss: 0.9899 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0531 - accuracy: 0.9781 - val_loss: 0.9711 - val_accuracy: 0.7664
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0467 - accuracy: 0.9785 - val_loss: 1.3505 - val_accuracy: 0.7598
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0433 - accuracy: 0.9812 - val_loss: 1.4044 - val_accuracy: 0.7651
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0432 - accuracy: 0.9807 - val_loss: 1.4923 - val_accuracy: 0.7717


In [70]:
# Make predictions with our bidirectional model
model_4_pred_probs = model_4.predict(x=val_sentences)
model_4_pred_probs

24/24 [==============================] - 1s 3ms/step


array([[3.74879921e-04],
       [5.89466095e-01],
       [9.99947548e-01],
       [2.30767697e-01],
       [2.97115675e-05],
       [9.99755323e-01],
       [9.55910623e-01],
       [9.99981880e-01],
       [9.99956489e-01],
       [9.98958588e-01],
       [5.60048029e-05],
       [9.93029416e-01],
       [7.14441485e-05],
       [2.39988640e-01],
       [3.10264331e-05],
       [7.75742461e-04],
       [5.09845995e-05],
       [4.39297437e-05],
       [3.34884599e-03],
       [9.99605954e-01],
       [9.99913573e-01],
       [4.96437096e-05],
       [9.99878645e-01],
       [3.74426338e-04],
       [9.99930620e-01],
       [9.99975801e-01],
       [1.27561856e-04],
       [3.77846765e-04],
       [4.53534703e-05],
       [5.82978427e-01],
       [9.99276936e-01],
       [5.73295809e-04],
       [3.35336267e-03],
       [1.60040523e-04],
       [9.16288137e-01],
       [3.06021184e-01],
       [9.99942541e-01],
       [1.52407497e-01],
       [2.84138834e-03],
       [9.99942660e-01],


In [71]:
# Convert pred probs to labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [72]:
# Calculate the results of our bidirectional model
model_4_results = evaluate_classification_metrics(y_true=val_labels,
                                                  y_pred=model_4_preds)
model_4_results

{'accuracy': 77.16535433070865,
 'precision': 0.7741380916586217,
 'recall': 0.7716535433070866,
 'f1': 0.7691811868378113}

In [73]:
np.array(list(model_4_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

Nous n'avons donc pas de meilleurs résultats avec model_4 comparé à model_0

In [74]:
np.array(list(model_4_results.values())) > np.array(list(model_3_results.values()))

array([False, False, False, False])

Et même moins bien que l'unidirectionnel

Video N°241: Discussing the intuition behind Conv1D neural networks for text and sequences

## Convolutional Neural Networks for text (and other types of sequences)

We've used CNN's for images but images are typically 2D (height x width)... however, our text data is 1D.

Previously we've Conv2D for our image data but now we're going to use Conv1D.

The typical structure of a Conv1D model for sequences (in our case, text):

```
Inputs (text) -> Tokenization -> Embedding -> Layer(s) (typically Conv1D + pooling) -> Outputs (class probabilities)
```

### Model 5: Conv1D

For different explanations of parameters see:
* https://poloclub.github.io/cnn-explainer/ (this is for 2D but can relate to 1D data)
* Difference between "same" and "valid" padding: 
https://stackoverflow.com/questions/37674306/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-t

    "VALID" = without padding:

       inputs:         1  2  3  4  5  6  7  8  9  10 11 (12 13)
                      |________________|                dropped
                                     |_________________|

    "SAME" = with zero padding:

                   pad|                                      |pad
       inputs:      0 |1  2  3  4  5  6  7  8  9  10 11 12 13|0  0
                   |________________|
                                  |_________________|
                                                 |________________|

In this example:

    Input width = 13
    Filter width = 6
    Stride = 5

Notes:

    "VALID" only ever drops the right-most columns (or bottom-most rows).
    "SAME" tries to pad evenly left and right, but if the amount of columns to be added is odd, it will add the extra column to the right, as is the case in this example (the same logic applies vertically: there may be an extra row of zeros at the bottom).

Edit:

About the name:

    With "SAME" padding, if you use a stride of 1, the layer's outputs will have the same spatial dimensions as its inputs.
    With "VALID" padding, there's no "made-up" padding inputs. The layer only uses valid input data.


In [75]:
# Test out our embedding layer, Conv1D layer and max pooling
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sequence into embedding 
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5, # this is also referred to as an ngram of 5 (meaning it looks at 5 words at a time)
                        strides=1, # default
                        activation="relu",
                        padding="same") # default = 'valid', the output is smaller than the input shape, 'same' means output is same shape as input
conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # equivalent to "get the most important feature" or "get the feature with the highest value"
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 32]), TensorShape([1, 32]))

In [76]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04570354, -0.00869273, -0.02861574, ...,  0.02163722,
         -0.02563227, -0.02270111],
        [-0.00309232,  0.0177151 ,  0.03353585, ...,  0.00463525,
          0.01992956,  0.02701294],
        [-0.05344785, -0.06103592, -0.00085171, ..., -0.05631393,
         -0.02102445, -0.01649631],
        ...,
        [-0.00958463, -0.01181003, -0.02606721, ..., -0.0359796 ,
         -0.04120681,  0.0043028 ],
        [-0.00958463, -0.01181003, -0.02606721, ..., -0.0359796 ,
         -0.04120681,  0.0043028 ],
        [-0.00958463, -0.01181003, -0.02606721, ..., -0.0359796 ,
         -0.04120681,  0.0043028 ]]], dtype=float32)>

In [77]:
conv_1d_output

<tf.Tensor: shape=(1, 15, 32), dtype=float32, numpy=
array([[[0.05387969, 0.0516783 , 0.04726673, 0.03816786, 0.        ,
         0.        , 0.04774302, 0.02815962, 0.        , 0.        ,
         0.        , 0.02932187, 0.        , 0.0203962 , 0.04313956,
         0.01688748, 0.009301  , 0.        , 0.        , 0.        ,
         0.0034993 , 0.        , 0.0605546 , 0.        , 0.01031689,
         0.00852167, 0.        , 0.03018137, 0.        , 0.0445616 ,
         0.        , 0.01803041],
        [0.        , 0.        , 0.02614683, 0.        , 0.        ,
         0.0279519 , 0.03192424, 0.        , 0.02874375, 0.03538559,
         0.00124985, 0.02016617, 0.        , 0.03323073, 0.        ,
         0.        , 0.02298411, 0.02971178, 0.07530764, 0.01494023,
         0.        , 0.        , 0.0107155 , 0.        , 0.01547737,
         0.        , 0.        , 0.01699599, 0.        , 0.08412334,
         0.        , 0.02191723],
        [0.        , 0.        , 0.07772033, 0.    

In [78]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.05387969, 0.05423424, 0.07772033, 0.03816786, 0.02242459,
        0.08155822, 0.04774302, 0.06794797, 0.07752734, 0.05042065,
        0.0113781 , 0.06429335, 0.        , 0.03623125, 0.04313956,
        0.01688748, 0.04892004, 0.03333205, 0.07530764, 0.08063727,
        0.01010844, 0.03291697, 0.09573942, 0.01764555, 0.0646664 ,
        0.02676195, 0.        , 0.03018137, 0.05604574, 0.08412334,
        0.0196848 , 0.02191723]], dtype=float32)>

Video N°242: Model 5: Building, fitting and evaluating a 1D CNN for text

In [79]:
# Create 1-dimensional convolutional layer to model sequences
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64,
                  kernel_size=5,
                  strides=1,
                  activation="relu",
                  padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(units=64, activation='relu')(x)
outputs = layers.Dense(units=1, activation='sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

# Compile Conv1D
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

# Get a summary of our Conv1D model
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Gl  (None, 64)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_4 (Dense)             (None, 1)              

In [80]:
# Fit the model
model_5_history = model_5.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20240426-120418
Epoch 1/5
215/215 [==============================] - 9s 35ms/step - loss: 0.1228 - accuracy: 0.9597 - val_loss: 0.9204 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0744 - accuracy: 0.9721 - val_loss: 1.1438 - val_accuracy: 0.7585
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0627 - accuracy: 0.9747 - val_loss: 1.1700 - val_accuracy: 0.7520
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0558 - accuracy: 0.9766 - val_loss: 1.2160 - val_accuracy: 0.7533
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0513 - accuracy: 0.9778 - val_loss: 1.2842 - val_accuracy: 0.7612


In [81]:
# Make some predictions with our Conv1D model
model_5_pred_probs = model_5.predict(x=val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[2.0913078e-02],
       [8.9808482e-01],
       [9.9981016e-01],
       [7.8126900e-02],
       [3.0668562e-07],
       [9.9648285e-01],
       [8.6458319e-01],
       [9.9997294e-01],
       [9.9999905e-01],
       [9.2240685e-01]], dtype=float32)

In [82]:
# Convert model 5 pred probs to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [83]:
# Calculate results for model 5
model_5_results = evaluate_classification_metrics(y_true=val_labels,
                                                  y_pred=model_5_preds)
model_5_results

{'accuracy': 76.11548556430446,
 'precision': 0.7618883943071393,
 'recall': 0.7611548556430446,
 'f1': 0.7593763358258424}

In [84]:
np.array(list(model_5_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

Nous n'avons donc pas de meilleurs résultats avec model_5 comparé à model_0

Video N°243: Using TensorFlow Hub for pretrained word embeddings (transfer learning for NLP)

## Model 6: TensorFlow Hub Pretrained Sentence Encoder

Now we've built a few of our own model's, let's try and use transfer learning for NLP, specifically using TensorFlow Hub's Universal Sentence Encoder:
- [https://tfhub.dev/google/universal-sentence-encoder/4](https://tfhub.dev/google/universal-sentence-encoder/4)
- [https://www.kaggle.com/models/google/universal-sentence-encoder](https://www.kaggle.com/models/google/universal-sentence-encoder)

See how the USE was created here: [https://arxiv.org/abs/1803.11175](https://arxiv.org/abs/1803.11175)

In [85]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence, "When you can the universal sentence encoder on a sentence, it turns it into numbers."])
print(embed_samples[0][:50])

tf.Tensor(
[-0.01157027  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680985  0.05589838 -0.01068729 -0.00597292  0.00639323 -0.0181952
  0.00030814  0.09105888  0.05874645 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865346 -0.04209305  0.0267898   0.03011008  0.00321069
 -0.00337971 -0.04787356  0.02266719 -0.00985925 -0.04063613 -0.01292093
 -0.04666384  0.056303   -0.03949255  0.00517688  0.02495828 -0.07014441
  0.02871508  0.04947684 -0.00633978 -0.08960193  0.02807117 -0.00808362
 -0.01360601  0.0599865  -0.10361787 -0.05195374  0.00232955 -0.0233253
 -0.03758105  0.03327729], shape=(50,), dtype=float32)


In [86]:
embed_samples[0].shape

TensorShape([512])

Video N°244: Model 6: Building, training and evaluating a transfer learning model for NLP

In [87]:
# Create a Keras Layer using the USE pretrained layer from tensorflow hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [88]:
# Create model using the Sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(units=64, activation='relu'),
    layers.Dense(units=1, activation='sigmoid', name='output_layer')
], name="model_6_USE")

# Compile
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_5 (Dense)             (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [89]:
# Train a classifier on top of USE pretrained embeddings
model_6_history = model_6.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20240426-120551
Epoch 1/5
215/215 [==============================] - 5s 15ms/step - loss: 0.5039 - accuracy: 0.7882 - val_loss: 0.4586 - val_accuracy: 0.7953
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4154 - accuracy: 0.8156 - val_loss: 0.4422 - val_accuracy: 0.8045
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4022 - accuracy: 0.8189 - val_loss: 0.4442 - val_accuracy: 0.8084
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3938 - accuracy: 0.8215 - val_loss: 0.4395 - val_accuracy: 0.8071
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3871 - accuracy: 0.8307 - val_loss: 0.4299 - val_accuracy: 0.8110


In [90]:
# Make predictions with USE TF Hub model
model_6_pred_probs = model_6.predict(x=val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 9ms/step


array([[0.22077943],
       [0.82412356],
       [0.9881435 ],
       [0.20356844],
       [0.80163777],
       [0.75716734],
       [0.9810457 ],
       [0.97982556],
       [0.9424449 ],
       [0.1127803 ]], dtype=float32)

In [91]:
# Convert prediction probabilities to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1.,
       1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [92]:
# Calculate model 6 performance metrics
model_6_results = evaluate_classification_metrics(y_true=val_labels,
                                                  y_pred=model_6_preds)
model_6_results

{'accuracy': 81.10236220472441,
 'precision': 0.8118674741067483,
 'recall': 0.8110236220472441,
 'f1': 0.8099858139755507}

In [93]:
np.array(list(model_6_results.values())) > np.array(list(baseline_results.values()))

array([ True,  True,  True,  True])

In [94]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

Video N°245: Preparing subsets of data for model 7 (same as model 6 but 10% of data)
Modified during N°247

## Model 7: TF Hub Pretrained USE but with 10% of training data

Transfer learning really helps when you don't have a large dataset.

To see how our model performs on a smaller dataset, let's replicate `model_6` except we'll train it on 10% of the data

In [95]:
# ## Noote: Making data splits like below leads to data leakage (model_7 trained on 10% data, outperforms model_6 trained on 100% data)
# ## DO NOT make data splits which leak data from validation/test sets into training set

# # Create subsets of 10% of the training data
# train_10_percent = train_df_shuffled[["text", "target"]].sample(frac=0.1, random_state=42)
# train_sentences_10_percent = train_10_percent["text"].to_list()
# train_labels_10_percent = train_10_percent["target"].to_list()
# len(train_sentences_10_percent), len(train_labels_10_percent)

(761, 761)

> 🔑 **Note:** Be *very* careful when creating training/val/test splits that you don't leak data across the datasets, otherwise your model evaluation metrics will be wrong. If something looks too good to be true (a model trained on 10% of data outperforming the same model trained on 100% of data) trust your gut and go back through to find where the error may lie.

In [111]:
# Making a better dataset split (no data leakage)
number_of_rows = int(0.1 * len(train_sentences))
train_sentences_10_percent = train_sentences[:number_of_rows]
train_labels_10_percent = train_labels[:number_of_rows]

In [112]:
# Check the number of each label in the updated training data subset
pd.Series(np.array(train_labels_10_percent)).value_counts()

0    406
1    279
Name: count, dtype: int64

In [113]:
_# Check the number of targets in our subset of data
# train_10_percent["target"].value_counts()

7296

In [114]:
train_df_shuffled["target"].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

Video N°246: Model 7: Building, training and evaluating a transfer learning model on 10% data

To recreate a model the same as a previous model you've created you can use the `tf.keras.models.clone_model()` method, see more here: [https://www.tensorflow.org/api_docs/python/tf/keras/models/clone_model](https://www.tensorflow.org/api_docs/python/tf/keras/models/clone_model)

In [115]:
# Let's build a model the same as model_6
# model_7 = tf.keras.models.clone_model(model_6)
# Problème: le nom du modèle reste "model_6_USE, pour éviter des erreurs d'interprétations nous recréons model_7:
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(units=64, activation='relu'),
    layers.Dense(units=1, activation='sigmoid', name='output_layer')
], name="model_7_USE")
# Compile model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary
model_7.summary()

Model: "model_7_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_7 (Dense)             (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [116]:
# Fit the model to the 10% training data subsets
model_7_history = model_7.fit(x=train_sentences_10_percent,
                              y=train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="tf_hub_sentence_encoder_10_percent_correct_split")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder_10_percent_correct_split/20240426-133401
Epoch 1/5
22/22 [==============================] - 2s 43ms/step - loss: 0.6730 - accuracy: 0.6628 - val_loss: 0.6507 - val_accuracy: 0.7100
Epoch 2/5
22/22 [==============================] - 0s 21ms/step - loss: 0.6057 - accuracy: 0.8044 - val_loss: 0.5980 - val_accuracy: 0.7546
Epoch 3/5
22/22 [==============================] - 0s 23ms/step - loss: 0.5341 - accuracy: 0.8190 - val_loss: 0.5442 - val_accuracy: 0.7625
Epoch 4/5
22/22 [==============================] - 0s 22ms/step - loss: 0.4736 - accuracy: 0.8263 - val_loss: 0.5081 - val_accuracy: 0.7848
Epoch 5/5
22/22 [==============================] - 0s 22ms/step - loss: 0.4289 - accuracy: 0.8350 - val_loss: 0.4924 - val_accuracy: 0.7861


In [117]:
# Make predictions with the model trained on 10% of the data
model_7_pred_probs = model_7.predict(x=val_sentences)
model_7_pred_probs[:10]

24/24 [==============================] - 0s 8ms/step


array([[0.2280759 ],
       [0.56966096],
       [0.89314824],
       [0.3671889 ],
       [0.53349465],
       [0.6716129 ],
       [0.8713287 ],
       [0.79482704],
       [0.8286645 ],
       [0.17486477]], dtype=float32)

In [118]:
# Turn pred probs into labels
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0.,
       1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [119]:
# Evaluating model 7 predictions
model_7_results = evaluate_classification_metrics(y_true=val_labels,
                                                  y_pred=model_7_preds)
model_7_results

{'accuracy': 78.60892388451444,
 'precision': 0.7870194413148897,
 'recall': 0.7860892388451444,
 'f1': 0.7846591268156005}

In [120]:
np.array(list(model_7_results.values())) > np.array(list(model_6_results.values()))

array([False, False, False, False])

Video N°247: Fixing our data leakage issue with model 7 and retraining it
Video N°248: Comparing all our modelling experiments evaluation metrics